# Imports 

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np

from datasets import load_dataset
from datasets import Value, ClassLabel, Features, DatasetDict


import transformers
from transformers import AutoTokenizer
import torch

from preprocessing.cleaning_utils import *

In [3]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cuda')

# Read in Data

In [4]:
mimic_dir = "/home/vs428/project/MIMIC/files/mimiciii/1.4/"
n2c2_dir = "/home/vs428/project/n2c2/2022/N2C2-AP-Reasoning/"
n2c2_data_dir =  "/home/vs428/project/n2c2/2022/Data/"


In [5]:
classes = ['Not Relevant', 'Neither', 'Indirect', 'Direct']
features = Features({
    'ROW ID':Value("int64"),
    'HADM ID':Value("int64"),
    'Assessment':Value("string"),
    'Plan Subsection':Value("string"),
    "Relation":Value("string")
}) 

dataset = load_dataset("csv", data_files={
                            "train":n2c2_data_dir + "train.csv",
                            "valid":n2c2_data_dir + "dev.csv",
                        },

                       features=features)
# dataset = dataset.class_encode_column("Relation")
dataset = dataset.rename_column("Relation", "label")

Using custom data configuration default-b1948d86214b7517
Reusing dataset csv (/home/vs428/.cache/huggingface/datasets/csv/default-b1948d86214b7517/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


  0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
dataset

DatasetDict({
    train: Dataset({
        features: ['ROW ID', 'HADM ID', 'Assessment', 'Plan Subsection', 'label'],
        num_rows: 4633
    })
    valid: Dataset({
        features: ['ROW ID', 'HADM ID', 'Assessment', 'Plan Subsection', 'label'],
        num_rows: 597
    })
})

In [7]:
notes = pd.read_csv(mimic_dir + "NOTEEVENTS.csv")

/tmp/ipykernel_95491/1268382413.py:1: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  notes = pd.read_csv(mimic_dir + "NOTEEVENTS.csv")


In [8]:
notes = notes.sort_values("CHARTTIME")

In [21]:
print(notes[notes['ROW_ID'] == 553008]['TEXT'].values[0])

Chief Complaint:  hypoxia/respiratory failure
   HPI:
   69F afib on coumadin, COPD, CRI, stroke, who presented to [**Hospital1 709**] on
   [**2178-2-3**] with black stool and was found to have hct of 23 in the setting
   of INR 11. She was transfused 3U pRBCs and 4U FFP and was given vitamin
   k. The patient was put on PPI drip for concern for possible upper GI
   source and was taken for EGD though in the GI suite was developed
   hypoxemia and the procedure was aborted. She also required rapid
   response team 1/22pm for rapid afib to 150s and was having periodic
   hypoxia as well. She was treated with IV dilt and PO dilt was increased
   and she received a digoxin load of 0.5 as well. For the hypoxemia this
   was thought [**2-16**] COPD and rapid afib. Dr. [**Last Name (STitle) 3598**] has kept the patient
   on the floor and is confident that patient is stable for floor at
   [**Hospital1 1**]. The hct is now bumped to 37.5 after the 3U pRBCs and the patient
   has reportedly 

In [27]:
HADMID = 195768

In [51]:
test = notes[(notes['HADM_ID'] == HADMID) & (notes['DESCRIPTION'].str.contains("Physician") & (notes['DESCRIPTION'].str.contains("Note")))
     ]

In [64]:
test_text = r"and there was discussion about intubating her, however she was strongly"
display(test[~test['TEXT'].str.contains(test_text)]), display(test[test['TEXT'].str.contains(test_text)])


,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
548444,553008,55440,195768.0,2178-02-06,2178-02-06 16:45:00,2178-02-06 16:45:51,Physician,Physician Resident Admission Note,18767.0,NaN,Chief Complaint: hypoxia/respiratory failure\...
548511,553014,55440,195768.0,2178-02-06,2178-02-06 17:05:00,2178-02-06 17:05:21,Physician,Physician Attending Admission Note - MICU,17419.0,NaN,Chief Complaint: 69 yr woamn with afib on cou...
548595,553116,55440,195768.0,2178-02-07,2178-02-07 08:42:00,2178-02-07 08:43:20,Physician,Physician Attending Progress Note,17419.0,NaN,Chief Complaint: Pt continued hypoxemic. Refus...
550788,553225,55440,195768.0,2178-02-08,2178-02-08 06:40:00,2178-02-08 10:27:27,Physician,Physician Resident Progress Note,16003.0,NaN,TITLE:\n Chief Complaint: Respiratory Distre...
550929,553222,55440,195768.0,2178-02-08,2178-02-08 09:34:00,2178-02-08 09:34:22,Physician,Physician Attending Progress Note,17419.0,NaN,Chief Complaint:\n HPI:\n 24 Hour Events:\...
550797,553234,55440,195768.0,2178-02-08,2178-02-08 12:39:00,2178-02-08 12:40:06,Physician,Cardiology Comprehensive Physician Note,19996.0,NaN,TITLE:\n Date of service: [**2178-2-8**]\n ...
548806,553371,55440,195768.0,2178-02-09,2178-02-09 05:46:00,2178-02-09 10:37:10,Physician,Physician Resident Progress Note,19680.0,NaN,TITLE:\n Chief Complaint:\n 24 Hour Events...
548786,553345,55440,195768.0,2178-02-09,2178-02-09 05:46:00,2178-02-09 07:48:13,Physician,Physician Resident Progress Note,19680.0,NaN,TITLE:\n Chief Complaint:\n 24 Hour Events...
548952,553325,55440,195768.0,2178-02-09,2178-02-09 05:46:00,2178-02-09 05:46:52,Physician,Physician Resident Progress Note,19680.0,NaN,TITLE:\n Chief Complaint:\n 24 Hour Events...
548797,553362,55440,195768.0,2178-02-09,2178-02-09 09:52:00,2178-02-09 09:52:17,Physician,Physician Attending Progress Note,16383.0,NaN,Chief Complaint: GI Bleed\n HYpoxia\n Atri...


,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
548577,553098,55440,195768.0,2178-02-07,2178-02-07 07:04:00,2178-02-07 07:05:04,Physician,Physician Resident Progress Note,18414.0,NaN,TITLE:\n Chief Complaint:\n 24 Hour Events...
548587,553108,55440,195768.0,2178-02-07,2178-02-07 07:04:00,2178-02-07 07:50:07,Physician,Physician Resident Progress Note,18414.0,NaN,TITLE:\n Chief Complaint:\n 24 Hour Events...
548602,553123,55440,195768.0,2178-02-07,2178-02-07 07:04:00,2178-02-07 09:38:55,Physician,Physician Resident Progress Note,18414.0,NaN,TITLE:\n Chief Complaint:\n 24 Hour Events...
548605,553126,55440,195768.0,2178-02-07,2178-02-07 07:04:00,2178-02-07 09:46:06,Physician,Physician Resident Progress Note,18414.0,NaN,TITLE:\n Chief Complaint:\n 24 Hour Events...
548521,553207,55440,195768.0,2178-02-08,2178-02-08 06:40:00,2178-02-08 06:40:19,Physician,Physician Resident Progress Note,16003.0,NaN,TITLE:\n Chief Complaint: Respiratory Distre...


(None, None)

In [82]:
test[test['TEXT'].str.contains(test_text)]['ROW_ID'].values

array([553098, 553108, 553123, 553126, 553207])

In [47]:
# print(test.loc[548595]['TEXT'])
# is the row/adm id from the dataset, which doesn't match the text
notes[(notes["ROW_ID"] == 553008) & (notes["HADM_ID"] == 195768)]
# print(notes[(notes["ROW_ID"] == 553008) & (notes["HADM_ID"] == 195768)]['TEXT'].values[0])

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
548444,553008,55440,195768.0,2178-02-06,2178-02-06 16:45:00,2178-02-06 16:45:51,Physician,Physician Resident Admission Note,18767.0,NaN,Chief Complaint: hypoxia/respiratory failure\...


In [50]:
print(notes[(notes["ROW_ID"] == 553008) & (notes["HADM_ID"] == 195768)]['TEXT'].values[0])

Chief Complaint:  hypoxia/respiratory failure
   HPI:
   69F afib on coumadin, COPD, CRI, stroke, who presented to [**Hospital1 709**] on
   [**2178-2-3**] with black stool and was found to have hct of 23 in the setting
   of INR 11. She was transfused 3U pRBCs and 4U FFP and was given vitamin
   k. The patient was put on PPI drip for concern for possible upper GI
   source and was taken for EGD though in the GI suite was developed
   hypoxemia and the procedure was aborted. She also required rapid
   response team 1/22pm for rapid afib to 150s and was having periodic
   hypoxia as well. She was treated with IV dilt and PO dilt was increased
   and she received a digoxin load of 0.5 as well. For the hypoxemia this
   was thought [**2-16**] COPD and rapid afib. Dr. [**Last Name (STitle) 3598**] has kept the patient
   on the floor and is confident that patient is stable for floor at
   [**Hospital1 1**]. The hct is now bumped to 37.5 after the 3U pRBCs and the patient
   has reportedly 

In [79]:
test.loc[[548521]]

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
548595,553116,55440,195768.0,2178-02-07,2178-02-07 08:42:00,2178-02-07 08:43:20,Physician,Physician Attending Progress Note,17419.0,NaN,Chief Complaint: Pt continued hypoxemic. Refus...


In [80]:
print(test.at[548521, 'TEXT'])

Chief Complaint: Pt continued hypoxemic. Refused BiPAP and intubation.
   Found to have NSTEMI. Remained tachycardic.
   HPI:
   Started on Amiodarone. Had a-line; required neo for BP support
   transiently.
   24 Hour Events:
 MULTI LUMEN - START [**2178-2-6**] 04:04 PM
 ARTERIAL LINE - START [**2178-2-6**] 07:19 PM
   No further GI bleeding. ABG last  7.54/27/63
                                                         7.52/33/91 on
   repeat
   Allergies:
   Codeine
   Unknown;
   Azithromycin
   Unknown;
   Morphine
   hallucinations;
   Last dose of Antibiotics:
   Levofloxacin - [**2178-2-6**] 06:03 PM
   Infusions:
   Amiodarone - 0.5 mg/min
   Furosemide (Lasix) - 1 mg/hour
   Other ICU medications:
   Furosemide (Lasix) - [**2178-2-6**] 04:52 PM
   Metoprolol - [**2178-2-6**] 06:15 PM
   Other medications:
   fluticasone/salmeterol, citalopram, atorvastatin, tiotropium, docusate,
   RISS, levoflox, amiodarone, metoprolol, lasix, xopenex
   Changes to medical and family history:

In [57]:
# split_leading_symptom_list(dataset['train'])
test = dataset['train'].map(split_leading_symptom_list)

  0%|          | 0/4633 [00:00<?, ?ex/s]

In [58]:
vals = [553008, 553008, 553008, 553008, 553008, 553008, 553008, 553008, 553008, 738065, 738065, 738065, 738065, 445696, 445696, 445696, 445696, 445696, 445696, 445696, 445696, 436272, 436272, 436272, 436272, 436272, 436272, 436272, 436272, 331191, 331191, 331191, 331191, 331191, 331191, 545421, 545421, 545421, 545421, 545421, 545421, 545421, 545421, 545421, 545421, 676895, 676895, 676895, 676895, 676895, 676895, 676895, 676895, 676895, 676895, 728137, 728137, 728137, 728137, 728137, 728137, 601920, 601920, 601920, 601920, 601920, 601920, 601920, 601920, 383898, 383898, 383898, 383898, 383898, 722639, 722639, 722639, 722639, 651964, 651964, 651964, 651964, 651964, 651964, 651964, 651964, 475506, 475506, 475506, 475506, 475506, 475506, 475506, 475506, 475506, 528840, 528840, 528840, 528840, 528840, 528840, 528840, 709565, 709565, 709565, 709565, 709565, 709565, 709565, 691484, 691484, 691484, 691484, 691484, 691484, 691484, 691484, 691484, 691484, 691484, 691484, 392428, 392428, 392428, 392428, 392428, 392428, 392428, 392428, 392428, 392428, 392428, 587245, 587245, 587245, 587245, 587245, 587245, 587245, 500976, 500976, 500976, 500976, 500976, 500976, 500976, 500976, 500976, 500976, 500976, 500976, 500976, 500976, 500976, 419302, 419302, 419302, 419302, 419302, 419302, 419302, 419302, 419302, 419302, 419302, 419302, 419302, 419302, 716756, 716756, 716756, 716756, 392428, 392428, 392428, 392428, 392428, 392428, 392428, 392428, 392428, 392428, 392428, 381740, 381740, 381740, 587504, 587504, 587504, 587504, 587504, 587504, 587504, 587504, 684511, 684511, 684511, 684511, 684511, 684511, 684511, 684511, 684511, 425604, 425604, 425604, 425604, 425604, 425604, 425604, 425604, 425604, 602755, 602755, 602755, 603638, 603638, 603638, 603638, 603638, 683418, 683418, 683418, 683418, 683418, 683418, 515939, 515939, 515939, 515939, 323002, 323002, 323002, 323002, 323002, 468825, 468825, 468825, 468825, 468825, 468825, 468825, 468825, 468825, 385051, 385051, 385051, 385051, 385051, 385051, 385051, 587904, 587904, 587904, 587904, 587904, 587904, 587904, 587904, 522276, 522276, 522276, 522276, 522276, 522276, 522276, 522276, 522276, 697113, 697113, 697113, 697113, 697113, 697113, 697113, 651352, 651352, 651352, 651352, 651352, 651352, 651352, 651352, 651352, 651352, 350647, 350647, 350647, 350647, 350647, 350647, 391658, 391658, 391658, 391658, 391658, 391658, 391658, 391658, 386536, 386536, 386536, 386536, 386536, 386536, 521543, 521543, 521543, 521543, 521543, 521543, 521543, 623363, 623363, 623363, 623363, 623363, 623363, 623363, 623363, 623363, 623363, 623363, 623363, 623363, 725786, 725786, 725786, 725786, 725786, 725786, 725786, 725786, 725786, 725786, 725786, 725786, 725786, 725786, 472449, 472449, 472449, 472449, 472449, 472449, 483223, 483223, 483223, 483223, 483223, 549605, 549605, 549605, 549605, 543714, 543714, 543714, 543714, 454462, 454462, 454462, 454462, 454462, 454462, 710529, 710529, 710529, 710529, 710529, 710529, 710529, 402570, 402570, 402570, 402570, 402570, 402570, 402570, 402570, 402570, 402570, 389345, 389345, 389345, 389345, 389345, 389345, 393810, 393810, 393810, 393810, 393810, 393810, 359152, 359152, 359152, 359152, 359152, 359152, 359152, 359152, 359152, 359152, 359152, 429714, 429714, 429714, 429714, 429714, 429714, 429714, 429714, 429714, 547209, 547209, 547209, 547209, 547209, 547209, 547209, 547209, 547209, 646933, 646933, 646933, 646933, 646933, 646933, 646933, 542822, 542822, 542822, 542822, 716755, 716755, 716755, 716755, 716755, 716755, 706685, 706685, 706685, 706685, 706685, 706685, 706685, 706685, 706685, 706685, 706685, 706685, 706685, 706685, 431903, 431903, 431903, 431903, 431903, 431903, 431903, 386165, 386165, 386165, 386165, 386165, 386165, 386165, 386165, 344733, 344733, 344733, 344733, 344733, 344733, 344733, 344733, 344733, 344733, 344733, 344733, 587974, 587974, 587974, 587974, 587974, 587974, 587974, 585163, 585163, 585163, 585163, 585163, 590045, 590045, 590045, 590045, 590045, 590045, 590045, 651849, 651849, 651849, 442077, 442077, 442077, 349536, 349536, 349536, 349536, 349536, 349536, 588595, 588595, 588595, 588595, 588595, 588595, 588595, 588595, 588595, 588595, 588595, 720279, 720279, 720279, 720279, 720279, 469323, 469323, 469323, 469323, 469323, 679519, 679519, 706663, 706663, 706663, 706663, 706663, 706663, 706663, 706663, 493722, 493722, 493722, 493722, 493722, 493722, 493722, 493722, 493722, 493722, 493722, 493722, 493722, 695360, 695360, 695360, 695360, 695360, 695360, 695360, 367073, 367073, 367073, 367073, 367073, 367073, 367073, 367073, 367073, 604658, 604658, 604658, 604658, 604658, 604658, 696106, 696106, 696106, 696106, 696106, 696106, 696106, 690114, 690114, 690114, 690114, 690114, 690114, 383076, 383076, 383076, 383076, 383076, 383076, 383076, 383076, 479599, 479599, 479599, 479599, 479599, 479599, 479599, 479599, 479599, 479599, 479599, 479599, 479599, 479599, 479599, 479599, 327329, 327329, 327329, 327329, 327329, 592789, 592789, 592789, 592789, 592789, 592789, 689023, 689023, 689023, 689023, 689023, 689023, 689023, 689023, 689023, 689023, 534926, 534926, 534926, 534926, 534926, 534926, 472408, 472408, 472408, 472408, 472408, 472408, 472408, 472408, 472408, 472408, 472408, 534108, 534108, 534108, 534108, 534108, 650610, 650610, 650610, 650610, 650610, 650610, 650610, 438035, 438035, 438035, 438035, 438035, 438035, 438035, 344270, 344270, 344270, 344270, 344270, 344270, 344270, 344270, 353433, 353433, 353433, 353433, 353433, 353433, 568839, 568839, 568839, 568839, 568839, 568839, 568839, 568839, 568839, 568839, 568839, 568839, 568839, 568839, 568839, 568839, 568839, 550276, 550276, 550276, 550276, 550276, 550276, 550276, 550276, ]

In [65]:
print("num with leading symptoms\nnum unique assessments\nratio")
print(len(set(vals))), print(len(set(dataset['train']['Assessment']))), print(len(set(vals))/len(set(dataset['train']['Assessment'])))

num with leading symptoms
num unique assessments
ratio
92
603
0.15257048092868988


(None, None, None)

In [113]:
# for data in test:
#     if data['Symptom List'] != None:
#         # print(data)
#         print(data['Assessment'])
#         print("\n---------------\n")
#         # break

In [116]:
already_seen = []
for data in dataset['train']:
    if data['ROW ID'] in set(vals) and data['ROW ID'] not in already_seen:
        print(data['Assessment'])
        already_seen.append(data['ROW ID'])
        print("\n---------------\n")
        

MYOCARDIAL INFARCTION, ACUTE (AMI, STEMI, NSTEMI)
 GASTROINTESTINAL BLEED, LOWER (HEMATOCHEZIA, BRBPR, GI BLEED, GIB)
 TACHYCARDIA, OTHER
 RESPIRATORY FAILURE, ACUTE (NOT ARDS/[**Doctor Last Name 2**])
   69F w/ h/o afib on coumadin, PUD, COPD, CKD, h/o CVA, HTN,
   hyperlipidemia transferred from [**Hospital1 709**] by family request after presenting
   with black stools, developed hypoxemia and tachyarrhythmia with RVR,
   now with continued respiratory distress complicated by demand ischemia.

---------------

Mr. [**Known lastname 12744**] is a 78-year-old patient of Dr. [**First Name8 (NamePattern2) 67**] [**Last Name (NamePattern1) 12743**], Dr.
   [**First Name8 (NamePattern2) 1862**] [**Last Name (NamePattern1) 7009**] and Dr. [**First Name4 (NamePattern1) **] [**Last Name (NamePattern1) 4425**] with atrial
   fibrillation/atrial tachycardia who was admitted for elective pulmonary
   vein isolation and is transferred to the CCU for 5 and 7 second pauses.

---------------

**Age

In [ ]:
# ask where these list of problems comes from
# should we be using the full note, the annotation seems to be that way? 